In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
train_data.shape #to see how the data looks
#train_data.tail()
#train_data.dtypes

(42000, 785)

In [3]:
test_data = pd.read_csv("/kaggle/input/digit-recognizer/test.csv")
test_data.head()
test_data.shape

(28000, 784)

In [4]:
train_X_1= train_data.drop(columns=['label'])
train_Y_1= train_data['label']
train_X_1.head(4)
train_X_1.shape
train_Y_1.shape

(42000,)

In [5]:
train_X_1 = np.array(train_X_1).reshape(train_X_1.shape[0], 28, 28, 1)
train_Y_1 = np.array(train_Y_1).reshape(train_Y_1.shape[0], 1)
test_data = np.array(test_data).reshape(test_data.shape[0], 28, 28, 1)

In [6]:
train_X_1 = train_X_1/ 255.0
test_data = test_data/ 255.0

In [7]:
import tensorflow as tf
from os import path, getcwd, chdir
#import numpy as np
from tensorflow import keras
import matplotlib.pyplot as plt

In [8]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy') > 0.99):
            print("\nReached 99% accuracy so cancelling training!")
            self.model.stop_training = True

In [9]:
"""
image_gen = ImageDataGenerator(
    #featurewise_center=True,
    #featurewise_std_normalization=True,
    rescale=1./255,
    rotation_range=15,
    width_shift_range=.15,
    height_shift_range=.15,
    horizontal_flip=True

image_gen.fit(train_X_1, augment=True)
"""

'\nimage_gen = ImageDataGenerator(\n    #featurewise_center=True,\n    #featurewise_std_normalization=True,\n    rescale=1./255,\n    rotation_range=15,\n    width_shift_range=.15,\n    height_shift_range=.15,\n    horizontal_flip=True\n\nimage_gen.fit(train_X_1, augment=True)\n'

In [10]:
"""
callbacks = myCallback()
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
#model.fit(train_X_1, train_Y_1, epochs=5, callbacks=[callbacks])
model.fit_generator(image_gen.flow(train_X_1,train_Y_1.toarray(), batch_size= 128),
          steps_per_epoch=  x_train.shape[0]//batch_size,
          epochs= 5,
          verbose=2)
pred = model.predict(test_data)

"""

"\ncallbacks = myCallback()\nmodel = tf.keras.models.Sequential([\n  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)),\n  tf.keras.layers.MaxPooling2D(2, 2),\n  tf.keras.layers.Conv2D(64, (3,3), activation='relu'),\n  tf.keras.layers.MaxPooling2D(2,2),\n  tf.keras.layers.Flatten(),\n  tf.keras.layers.Dense(128, activation='relu'),\n  tf.keras.layers.Dense(10, activation='softmax')\n])\nmodel.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])\nmodel.summary()\n#model.fit(train_X_1, train_Y_1, epochs=5, callbacks=[callbacks])\nmodel.fit_generator(image_gen.flow(train_X_1,train_Y_1.toarray(), batch_size= 128),\n          steps_per_epoch=  x_train.shape[0]//batch_size,\n          epochs= 5,\n          verbose=2)\npred = model.predict(test_data)\n\n"

In [11]:
#With Drop-out
callbacks = myCallback()
model = tf.keras.models.Sequential([
  tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(28, 28, 1)), #kernel_regularizer=regularizers.l2(0.0001)),
  tf.keras.layers.MaxPooling2D(2, 2),
  tf.keras.layers.Dropout(0.30),
    
  tf.keras.layers.Conv2D(64, (3,3), activation='relu'), #kernel_regularizer=regularizers.l2(0.0001)),
  tf.keras.layers.MaxPooling2D(2,2),
  tf.keras.layers.Dropout(0.30),
    
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),#kernel_regularizer=regularizers.l2(0.0001)),
  tf.keras.layers.Dropout(0.50),
    
  tf.keras.layers.Dense(10, activation='softmax')
])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.summary()
model.fit(train_X_1, train_Y_1, epochs=10, callbacks=[callbacks])
pred = model.predict(test_data)


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 13, 13, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 13, 13, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 11, 11, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0

In [12]:
pred2 = np.argmax(pred, axis=1)

In [13]:
pred_digits = pd.DataFrame({'ImageId': range(1,len(test_data)+1) ,'Label':pred2 })
pred_digits.to_csv("sample_submission.csv",index=False)
pred_digits.head()


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,9
4,5,3
